In [2]:

from datetime import datetime, timedelta
from collections import defaultdict
from dateutil.relativedelta import relativedelta
import json
import requests
def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d")

def get_month_date_range(target_month):
    start = datetime.strptime(target_month, "%Y-%m")
    end = (start + relativedelta(months=1)) - timedelta(days=1)
    return start, end

def get_overlap_range(start1, end1, start2, end2):
    latest_start = max(start1, start2)
    earliest_end = min(end1, end2)
    if latest_start > earliest_end:
        return None, None
    return latest_start, earliest_end

def days_inclusive(start, end):
    return (end - start).days + 1

def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    month_start, month_end = get_month_date_range(target_month)
    grouped = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        item_code = item.get("item_code")
        rate = float(item.get("rate", 0))
        qty = int(item.get("qty", 0))
        start_date = parse_date(item.get("start_date"))
        end_date = parse_date(item.get("end_date"))

        overlap_start, overlap_end = get_overlap_range(start_date, end_date, month_start, month_end)
        if not overlap_start:
            continue  # No overlap with target month

        active_days = days_inclusive(overlap_start, overlap_end)
        total_days_in_month = days_inclusive(month_start, month_end)
        proportion = active_days / total_days_in_month

        amount = rate * qty * proportion
        billing_period = f"{overlap_start.date()} to {overlap_end.date()}"
        key = (item_code, rate, billing_period)

        grouped[key]["qty"] += qty
        grouped[key]["amount"] += amount

    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), values in grouped.items():
        amount = round(values["amount"], 2)
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": amount,
            "billing_period": billing_period
        })
        total_revenue += amount

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }



item_list = [
    {
        "item_code": "Executive Desk (4*2)",
        "rate": 1080,
        "qty": 10,
        "start_date": "2024-10-15",
        "end_date": "2024-11-20"
    },
]



# Call the function for November 2024
# %%
bill = generate_monthly_bill(item_list, "2024-11")
# %%

print(json.dumps(bill, indent=2))



{
  "line_items": [
    {
      "item_code": "Executive Desk (4*2)",
      "rate": 1080.0,
      "qty": 10,
      "amount": 7200.0,
      "billing_period": "2024-11-01 to 2024-11-20"
    }
  ],
  "total_revenue": 7200.0
}
